Examining the output from the Files Stats Collector
files_packages.csv:  Contains 


In [ ]:
# This is an experimental file choser 
# import ipywidgets as widgets
# from IPython.display import display
# from IPython.display import clear_output
# 
# # Create a file upload widget
# file_upload = widgets.FileUpload(
#     accept='.csv',  # Specify the accepted file types (e.g., .csv)
#     multiple=False  # Set to True if you want to allow multiple file uploads
# )
# 
# # Display the file upload widget
# display(file_upload)
# 
# # Function to handle file upload
# def handle_upload(change):
#     uploaded_filename = list(file_upload.value.keys())[0]
#     uploaded_file = file_upload.value[uploaded_filename]['content']
#     
#     # You can process the uploaded file here, for example, by saving it or reading its content
#     # For demonstration purposes, let's print the content
#     clear_output()  # Clear the widget's output
#     print(f"Uploaded File Name: {uploaded_filename}")
#     print("File Content:")
#     print(uploaded_file.decode('utf-8'))  # Assuming it's a text file
# 
# # Attach the handle_upload function to the widget's change event
# file_upload.observe(handle_upload, names='value')


In [3]:
import pandas as pd

In [4]:

# Load the CSV files
files_packages = pd.read_csv('files_packages.csv')  # Columns: path, package
files_info = pd.read_csv('files_info.csv', delimiter='|')  # Columns: path, creation_time, access_time

In [5]:
# Merge dataframes on 'path' with an outer join to find unmatched files
merged_df = pd.merge(files_packages, files_info, on='path', how='outer', indicator=True)

# Report files in files_packages that are not in files_info
unreported_files_packages = merged_df[merged_df['_merge'] == 'left_only']
print("Files in files_packages not in files_info:")
print(unreported_files_packages[['path']])


Files in files_packages not in files_info:
                                                 path
0                                                  /.
1                                                  /.
2                                                  /.
3                                                  /.
4                                                  /.
...                                               ...
121158      /usr/share/doc/zlib1g-dev/examples/zran.c
121159      /usr/share/doc/zlib1g-dev/txtvsbin.txt.gz
121160                  /usr/share/man/man3/zlib.3.gz
121161              /usr/lib/x86_64-linux-gnu/libz.so
121162  /usr/share/doc/zlib1g-dev/changelog.Debian.gz

[105343 rows x 1 columns]


In [8]:
# Add 'directory' column to merged_df by extracting it from the 'path' column
merged_df['directory'] = merged_df['path'].apply(lambda x: '/'.join(x.split('/')[:-1]) if pd.notnull(x) else x)

# Filter out files that are not in files_packages
non_packaged_files = merged_df[merged_df['_merge'] == 'right_only']

# Group by directory and count files
directory_counts = non_packaged_files.groupby('directory').size()
print("\nDirectories with non-packaged files and their file counts:")
print(directory_counts)



Directories with non-packaged files and their file counts:
directory
/bin                         7
/boot                       10
/boot/grub                   5
/boot/grub/fonts             1
/boot/grub/i386-pc         269
                          ... 
/var/www/vems/viewer         6
/var/www/vems/viewer/js      3
/var/www/vems/vx            15
/var/www/vems/vx/js         33
/var/www/vems/xml            5
Length: 14315, dtype: int64


In [ ]:
# Merge the dataframes on 'path'
merged_df = pd.merge(files_packages, files_info, on='path')

# Group by 'package' and get the oldest 'access_time'
oldest_access_times = merged_df.groupby('package')['access_time'].min()
oldest_access_times

In [ ]:
oldest_access_times= oldest_access_times.to_frame()

In [ ]:

min_epoch_time = oldest_access_times['access_time'].min()
oldest_access_times['days_since_min_epoch'] = (oldest_access_times['access_time'] - min_epoch_time) /(24*60*60)
# Convert 'access_time' to datetime format
oldest_access_times['access_time'] = pd.to_datetime(oldest_access_times['access_time'], unit='s')

# Now you can use the .dt accessor

oldest_access_times['access_date'] = oldest_access_times['access_time'].dt.strftime('%Y-%m-%d')

# Format the 'access_time' column as YYYY-MM-DD


# Sort the packages by access time in ascending order
sorted_packages = oldest_access_times_reset = oldest_access_times.reset_index()
sorted_packages = sorted_packages.sort_values('access_time')

# Now, 'oldest_access_times_reset' contains the package names as a regular column


In [ ]:
# Exporting the DataFrame to a CSV file
sorted_packages.to_csv('oldest_access_times.csv', index=False)
